In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
from scipy import stats
import seaborn as sns
from functools import reduce
pd.options.mode.chained_assignment = None 
from statistics import mean
from profile_generation import *
from plotting import *
import warnings
warnings.simplefilter('ignore', np.RankWarning)
from scipy.optimize import curve_fit
import matplotlib.patches as patches
from scipy.optimize import curve_fit
from openpyxl import load_workbook
import math
from scipy.optimize import OptimizeWarning
warnings.simplefilter("ignore", OptimizeWarning)

In [3]:
# Generate profiles and join dataframes
patients_to_exclude_linear, patients_to_exclude_quad, list_of_patient_df, list_of_cal_pred_df, list_of_result_df = generate_profiles()
df, cal_pred, result_df = join_dataframes(list_of_patient_df, list_of_cal_pred_df, list_of_result_df)

# Print patients to exclude and ouput dataframes to excel as individual sheets
print_patients_to_exclude(patients_to_exclude_linear, patients_to_exclude_quad)
output_df_to_excel(df, cal_pred, result_df)

day_1    2.0
day_2    3.0
Name: 0, dtype: float64 3.5 4.0
[7.44026889e-05 8.62569933e-03]
day_1    2.0
day_2    3.0
Name: 0, dtype: float64 4.5 4.0
[0.0006152  0.02480314]
day_1    2.0
day_2    3.0
Name: 0, dtype: float64 5.5 4.0
[0.00235955 0.0485752 ]
day_1    2.0
day_2    3.0
Name: 0, dtype: float64 6.5 4.0
[0.00598424 0.07735789]
day_1    2.0
day_2    3.0
Name: 0, dtype: float64 7.5 4.0
[0.01184154 0.10881882]
day_1    2.0
day_2    3.0
Name: 0, dtype: float64 8.5 4.0
[0.01995565 0.14126447]
day_1    2.0
day_2    3.0
Name: 0, dtype: float64 9.5 4.0
[0.0301305  0.17358139]
day_1    2.0
day_2    3.0
Name: 0, dtype: float64 10.5 4.0
[0.04205938 0.20508384]
day_1    2.0
day_2    3.0
Name: 0, dtype: float64 11.5 4.0
[0.0554022  0.23537671]
day_1    2.0
day_2    3.0
Name: 0, dtype: float64 12.5 4.0
[0.06983045 0.26425451]
day_1    2.0
day_2    3.0
Name: 0, dtype: float64 13.5 4.0
[0.08504938 0.29163226]
day_1    2.0
day_2    3.0
Name: 0, dtype: float64 14.5 4.0
[0.10080612 0.3174998 ]
day

In [ ]:
dat = result_df.copy()
dat = dat[['deviation', 'method', 'patient', 'pred_day', 'response', 'prediction', 'half_life']]
dat['half_life'] = dat['half_life'].fillna('')
dat = dat.loc[(dat.method == 'Q_Cum_origin_dp_tau') | (dat.method == 'Q_Cum_origin_dp')]
dat['new_method'] = ""
for i in range(len(dat)):
    dat['new_method'].iloc[i] = dat['method'].iloc[i] + '_' + str(dat['half_life'].iloc[i])
    # print(dat['method'].iloc[i])
plot = sns.lineplot(data=dat, x="patient", y="deviation", hue="new_method", ci=None, legend=False)
plot = sns.lineplot(data=dat.loc[dat.method=='Q_Cum_origin_dp_ '], x="patient", y="deviation", color='b', ci=None, legend=False)
# dat.head()

# sns.lineplot(data=dat.loc[dat.patient==84], x="pred_day", y="deviation", hue="method")

# dat

In [ ]:
# Plotting
# perc_days_within_target_tac(result_df)
# perc_days_outside_target_tac(result_df)
# median_perc_within_acc_dev(result_df)
# can_benefit(result_df)
# modified_TTR(result_df)
# wrong_range(result_df)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import math

d = {'dose': [0.5, 1, 1.5, 1.5, 3, 3], 'response': [2.4, 2.8, 3.2, 3.1, 7.9, 10]}
df = pd.DataFrame(data=d)

# Calculate weight
j = 0
decay_weight = []
for i in range(len(df)):
    decay_weight.append(math.exp(-(24*(i))/(12/np.log(2))))

# Fit model
poly_reg = PolynomialFeatures(degree=2)
X = np.array(df.dose).reshape(-1, 1)
y = np.array(df.response)
X = poly_reg.fit_transform(X)
result = LinearRegression(fit_intercept=False).fit(X, y, decay_weight)
result.coef_
# new = 3
# prediction = result.predict(poly_reg.fit_transform([[new]]))[0]
# prediction

In [ ]:
np.array(3).reshape(-1, 1)

In [8]:
a = pd.DataFrame(columns=['col1','col2'])
a = a[0:0]
a.loc[2, :] = [1, 2]

b = pd.DataFrame(columns=['col1','col2'])
b = b[0:0]
b.loc[0, :] = [1, 2]
b.loc[1, :] = [1, 2]

pd.concat([a, b])

,col1,col2
2,1,2
0,1,2
1,1,2
